## POPULARITY OF TED TALKS (Decision tree using random forest algorithm)

Data source : Kaggle.com (webscraping from TED.com)
Link : https://www.kaggle.com/rounakbanik/ted-talks
Time period : 2006-2017
Number of records : 2550
Number of columns : 17
No of Unique identifier : 4


We are trying to predict the popularity of ted talks video using some features that we have identified important (preprocessing done separately) such as duration, languages and speaker categories. Out metric to decide the popularity of ted talk video is the total number of views. Total views for a video is available from the published date.
To compare views, normalization has to be done. Assumption is that total views follows an exponential distribution with expected mean = 15 days
Hence, Recommendation is to record views over time ( eg. Views in first week, Views in first month)

Hence Outcome- Success(>0.0003) or failure 

The dataset was preprocessed and cleaned before importing. 

### Importing data set :

In [49]:
import pandas as pd
df = pd.read_csv("C:/Users/susha/OneDrive/Desktop/latestdata.csv")
print(df)

               duration                 languages  Speaker.Categories  outcome
0     Between 10 and 20              More than 30       Author/Writer        0
1     Between 10 and 20              More than 30   Activist/Politics        0
2          More than 20         Between 21 and 30       Author/Writer        0
3     Between 10 and 20              More than 30   Activist/Politics        1
4     Between 10 and 20              More than 30   Activist/Politics        0
5          More than 20              More than 30          Healthcare        0
6     Between 10 and 20              More than 30    Performance Arts        1
7     Between 10 and 20  Less than or equal to 20  Architect/Designer        0
8          More than 20              More than 30    Science/Research        1
9          More than 20              More than 30       Author/Writer        1
10         More than 20         Between 21 and 30  Architect/Designer        0
11         More than 20  Less than or equal to 20   

In [50]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

from sklearn.model_selection import  cross_val_score

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split


### Defining features and outcome to train and test 

In [54]:
features = ["duration",  "languages",  "Speaker.Categories"]
df, targets = encode_target(df, "outcome")
y = df["Target"]
X = df[features]



In [55]:
from sklearn.model_selection import train_test_split

### Fitting the model

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.1)
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

le.fit(X_train['duration'].astype(str))
X_train['duration']=le.transform(X_train['duration'].astype(str))
X_test['duration']=le.transform(X_test['duration'].astype(str))

le.fit(X_train['languages'].astype(str))
X_train['languages']=le.transform(X_train['languages'].astype(str))
X_test['languages']=le.transform(X_test['languages'].astype(str))

le.fit(X_train['Speaker.Categories'].astype(str))
X_train['Speaker.Categories']=le.transform(X_train['Speaker.Categories'].astype(str))
X_test['Speaker.Categories']=le.transform(X_test['Speaker.Categories'].astype(str))




### Applying GridsearchCV and Random forest algorithm on Decision tree

In [62]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

dt = RandomForestClassifier()
# set of parameters to test
param_grid = {"criterion": ["gini", "entropy"],
              "max_depth": [2, 5, 10],
               "bootstrap" : [True,False],
               "n_estimators":[10,50,100]
               }

ts_gs = GridSearchCV(estimator=dt,param_grid=param_grid)
ts_gs.fit(X_train,y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [63]:
ts_gs.best_params_

{'bootstrap': True, 'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 50}

In [65]:
dt_bestparams = RandomForestClassifier(criterion='entropy',max_depth=5,bootstrap=False,n_estimators=50)


In [66]:
dt_bestparams.fit(X_train,y_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='entropy',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [67]:
predictions = dt_bestparams.predict(X_test)

In [68]:
from sklearn import metrics

### Result of Accuracy, Precision and Recall for our Decision tree

In [69]:
print("Accuracy based on 3 Features:",round(metrics.accuracy_score(y_test, predictions),4))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision based on 3 Features:",round(metrics.precision_score(y_test, predictions),3))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall based on 3 Features:",round(metrics.recall_score(y_test, predictions),3))

Accuracy based on 3 Features: 0.5992
Precision based on 3 Features: 0.565
Recall based on 3 Features: 0.793


In [70]:
print("Recall based on 3 Features:",round(metrics.f1_score(y_test, predictions),3))

Recall based on 3 Features: 0.66


In [72]:
y_train.shape

(2217,)

from sklearn.ensemble import RandomForestRegressor
import pydot
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3, random_state=42)
rf_small.fit(X_train, y_train)

# Extract the small tree
tree_small = rf_small.estimators_[5]

# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = features, rounded = True, precision = 1)

(graph, ) = pydot.graph_from_dot_file('small_tree.dot')

graph.write_png('small_tree.png')